## Project for rmrb - Part #[2]

In [31]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import gensim
import nltk

import os
import ast  # Convert string-represented list
import itertools
from collections import Counter  # For words counting

In [32]:
# Read in the corpus
rmrb = pd.read_csv("../../final_corpus_rmrb.csv", index_col=0)
rmrb.head()

,year,month,date,issue,column,date_title,title,author,text,tokenized_sent,token_para
64375,1965,1,1965-01-03,2,NaN,1965-01-03坚决破除迷信 改进原有设计,坚决破除迷信 改进原有设计,NaN,坚决破除迷信 改进原有设计洛阳轴承厂在严格的科学精神指导下，开展群众性的技术革新活动，轴承品...,"[['坚决', '破除迷信', '改进', '原有', '设计'], ['洛阳', '轴承厂...","['坚决 破除迷信 改进 原有 设计', '洛阳 轴承厂 严格 科学 精神 指导 开展 群众..."
64388,1965,1,1965-01-03,3,NaN,1965-01-03用新的斗争和新的成就迎接新的一年,用新的斗争和新的成就迎接新的一年,NaN,用新的斗争和新的成就迎接新的一年金日成首相号召朝鲜人民努力完成七年计划和加强反修斗争艾地主席...,"[['斗争', '成就', '迎接', '一年', '金日成', '首相', '号召', '...",['斗争 成就 迎接 一年 金日成 首相 号召 朝鲜 人民 努力完成 七年 计划 加强 反修...
64446,1965,1,1965-01-05,2,社论,1965-01-05革命的大会 民主的大会 团结的大会——庆祝第三届全国人民代表大会第一次会...,革命的大会 民主的大会 团结的大会——庆祝第三届全国人民代表大会第一次会议的伟大成就,NaN,革命的大会 民主的大会 团结的大会第三届全国人民代表大会第一次会议胜利闭幕了。会议听取了周恩...,"[['革命', '大会', '民主', '大会', '团结', '大会'], ['第三届',...","['革命 大会 民主 大会 团结 大会', '第三届 全国人民代表大会 第一次 会议 胜利 ..."
64573,1965,1,1965-01-08,4,NaN,1965-01-08美帝陷入各国人民四面八方包围之中 约翰逊反革命两手挽救不了失败命运,美帝陷入各国人民四面八方包围之中 约翰逊反革命两手挽救不了失败命运,NaN,朝越坦等国舆论抨击约翰逊国情咨文以和平词藻掩饰侵略野心美帝陷入各国人民四面八方包围之中 约...,"[['朝越', '坦等国', '舆论', '抨击', '约翰逊', '国情咨文', '和平'...","['朝越 坦等国 舆论 抨击 约翰逊 国情咨文 和平 词藻 掩饰 侵略 野心', '美帝 陷..."
64584,1965,1,1965-01-09,2,短评,1965-01-09创造力的解放,创造力的解放,NaN,卢王庄公社第一年种水稻，就得到了好收成。这件事情又一次雄辩地说明：社会主义革命是社会主义生产...,"[['卢王庄', '公社', '第一年', '水稻', '得到', '好收成'], ['这件...","['卢王庄 公社 第一年 水稻 得到 好收成', '这件 事情 一次 雄辩 说明 社会主义 ..."


In [135]:
# Function that converts string-represented nested list to list of tokens
def str2lst(string):
    article = []
    string.strip("]")
    strs = string.replace("[", "").split("],")
    for sent in strs:
        sentence = [word.strip().strip("]").strip("'").strip("'") for word in sent.split(",")]
        article.append(sentence)
    return article

In [164]:
rmrb["tokens_by_sent"] = rmrb["tokenized_sent"].apply(lambda x: str2lst(x))
rmrb.head()

,year,month,date,issue,column,date_title,title,author,text,tokenized_sent,token_para,tokens_by_sent
64375,1965,1,1965-01-03,2,NaN,1965-01-03坚决破除迷信 改进原有设计,坚决破除迷信 改进原有设计,NaN,坚决破除迷信 改进原有设计洛阳轴承厂在严格的科学精神指导下，开展群众性的技术革新活动，轴承品...,"[['坚决', '破除迷信', '改进', '原有', '设计'], ['洛阳', '轴承厂...","['坚决 破除迷信 改进 原有 设计', '洛阳 轴承厂 严格 科学 精神 指导 开展 群众...","[[坚决, 破除迷信, 改进, 原有, 设计], [洛阳, 轴承厂, 严格, 科学, 精神,..."
64388,1965,1,1965-01-03,3,NaN,1965-01-03用新的斗争和新的成就迎接新的一年,用新的斗争和新的成就迎接新的一年,NaN,用新的斗争和新的成就迎接新的一年金日成首相号召朝鲜人民努力完成七年计划和加强反修斗争艾地主席...,"[['斗争', '成就', '迎接', '一年', '金日成', '首相', '号召', '...",['斗争 成就 迎接 一年 金日成 首相 号召 朝鲜 人民 努力完成 七年 计划 加强 反修...,"[[斗争, 成就, 迎接, 一年, 金日成, 首相, 号召, 朝鲜, 人民, 努力完成, 七..."
64446,1965,1,1965-01-05,2,社论,1965-01-05革命的大会 民主的大会 团结的大会——庆祝第三届全国人民代表大会第一次会...,革命的大会 民主的大会 团结的大会——庆祝第三届全国人民代表大会第一次会议的伟大成就,NaN,革命的大会 民主的大会 团结的大会第三届全国人民代表大会第一次会议胜利闭幕了。会议听取了周恩...,"[['革命', '大会', '民主', '大会', '团结', '大会'], ['第三届',...","['革命 大会 民主 大会 团结 大会', '第三届 全国人民代表大会 第一次 会议 胜利 ...","[[革命, 大会, 民主, 大会, 团结, 大会], [第三届, 全国人民代表大会, 第一次..."
64573,1965,1,1965-01-08,4,NaN,1965-01-08美帝陷入各国人民四面八方包围之中 约翰逊反革命两手挽救不了失败命运,美帝陷入各国人民四面八方包围之中 约翰逊反革命两手挽救不了失败命运,NaN,朝越坦等国舆论抨击约翰逊国情咨文以和平词藻掩饰侵略野心美帝陷入各国人民四面八方包围之中 约...,"[['朝越', '坦等国', '舆论', '抨击', '约翰逊', '国情咨文', '和平'...","['朝越 坦等国 舆论 抨击 约翰逊 国情咨文 和平 词藻 掩饰 侵略 野心', '美帝 陷...","[[朝越, 坦等国, 舆论, 抨击, 约翰逊, 国情咨文, 和平, 词藻, 掩饰, 侵略, ..."
64584,1965,1,1965-01-09,2,短评,1965-01-09创造力的解放,创造力的解放,NaN,卢王庄公社第一年种水稻，就得到了好收成。这件事情又一次雄辩地说明：社会主义革命是社会主义生产...,"[['卢王庄', '公社', '第一年', '水稻', '得到', '好收成'], ['这件...","['卢王庄 公社 第一年 水稻 得到 好收成', '这件 事情 一次 雄辩 说明 社会主义 ...","[[卢王庄, 公社, 第一年, 水稻, 得到, 好收成], [这件, 事情, 一次, 雄辩,..."


In [154]:
# Function that flattens a nested list
def flatten(lst):
    return list(itertools.chain(*lst))

In [155]:
# Group the corpus by year
rmrb_yr = rmrb.groupby("year").agg({"tokens_by_sent": lambda x: x.sum()})
rmrb_yr["tokens"] = rmrb_yr["tokens_by_sent"].apply(lambda x: flatten(x))
rmrb_yr.head()

,tokens_by_sent,tokens
year,,
1965,"[[坚决, 破除迷信, 改进, 原有, 设计], [洛阳, 轴承厂, 严格, 科学, 精神,...","[坚决, 破除迷信, 改进, 原有, 设计, 洛阳, 轴承厂, 严格, 科学, 精神, 指导..."
1966,"[[一九六六年, 到来], [这是, 我国, 伟大, 第三个, 五年计划, 第一年], [我...","[一九六六年, 到来, 这是, 我国, 伟大, 第三个, 五年计划, 第一年, 我国, 各族..."
1967,"[[无产阶级, 文化大革命, 凯歌, 声中, 迎来, 一九六七年], [毛主席, 亲自, 发...","[无产阶级, 文化大革命, 凯歌, 声中, 迎来, 一九六七年, 毛主席, 亲自, 发动, ..."
1968,"[[伟大领袖, 毛主席, 一系列, 最新, 指示, 广大, 革命, 群众, 见面, 以后, ...","[伟大领袖, 毛主席, 一系列, 最新, 指示, 广大, 革命, 群众, 见面, 以后, 出..."
1969,"[[湖北省, 革命, 委员会, 遵照, 伟大领袖, 毛主席, 革命, 委员会, 实行, 一元...","[湖北省, 革命, 委员会, 遵照, 伟大领袖, 毛主席, 革命, 委员会, 实行, 一元化..."


In [148]:
# Save the tokens
for index, year in rmrb_yr.iterrows():
    path = f"../data/corpus_by_year/{index}.txt"
    with open(path, "w") as f:
        for sent in year["tokens_by_sent"]:
            f.writelines(word + " " for word in sent)
            f.write("\n")

## Counting words

In [165]:
# Construct a dictionary that stores the word counting 
n = 20
counters = {}
for index, piece in rmrb_yr.iterrows():
    count = Counter(piece["tokens"])
    sorted_count = sorted(count.items(), key=lambda x: x[1], reverse=True)
    counters[index] = sorted_count
counters[1989][:20]

[('我们', 2357),
 ('发展', 2037),
 ('社会主义', 1950),
 ('工作', 1589),
 ('问题', 1559),
 ('经济', 1535),
 ('中国', 1454),
 ('国家', 1259),
 ('社会', 1165),
 ('建设', 1162),
 ('人民', 1149),
 ('政治', 1025),
 ('一个', 960),
 ('改革', 947),
 ('我国', 825),
 ('坚持', 803),
 ('进行', 797),
 ('必须', 723),
 ('企业', 703),
 ('思想', 690)]

In [163]:
# Top n words
n = 30
top_n_words = {}
for k, v in counters.items():
    words = [pair[0] for pair in v[:n]]
    top_n_words[k] = words
pd.DataFrame(top_n_words)

,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002
0,人民,革命,革命,革命,革命,人民,我们,我们,我们,我们,...,发展,发展,发展,发展,发展,发展,发展,发展,发展,发展
1,我们,我们,无产阶级,毛主席,我们,我们,人民,人民,革命,革命,...,建设,建设,经济,中国,中国,经济,工作,工作,我们,建设
2,革命,人民,我们,无产阶级,无产阶级,革命,革命,革命,社会主义,斗争,...,经济,工作,企业,建设,经济,中国,中国,经济,经济,工作
3,他们,社会主义,毛主席,我们,毛主席,毛主席,毛主席,社会主义,无产阶级,历史,...,社会主义,经济,中国,经济,建设,工作,我们,我们,工作,中国
4,社会主义,他们,他们,群众,人民,斗争,社会主义,毛主席,路线,他们,...,我们,社会主义,建设,工作,社会主义,我们,经济,中国,中国,经济
5,斗争,斗争,人民,他们,社会主义,社会主义,他们,发展,人民,人民,...,工作,中国,问题,我们,我们,建设,企业,建设,建设,我们
6,发展,毛主席,斗争,人民,斗争,无产阶级,斗争,路线,毛主席,无产阶级,...,中国,企业,改革,企业,企业,国家,建设,社会,社会,重要
7,生产,无产阶级,资产阶级,毛泽东思想,他们,他们,发展,他们,发展,发展,...,问题,我们,工作,问题,工作,问题,问题,问题,问题,问题
8,进行,政治,资本主义,斗争,群众,发展,无产阶级,群众,他们,阶级,...,企业,问题,社会,国家,问题,企业,社会,企业,社会主义,社会
9,一个,修正主义,毛泽东思想,中国,一个,思想,问题,生产,群众,反动,...,改革,改革,我们,改革,国家,农村,我国,社会主义,人民,人民


In [166]:
counters

{1965: [('人民', 2975),
  ('我们', 2587),
  ('革命', 2398),
  ('他们', 1672),
  ('社会主义', 1561),
  ('斗争', 1375),
  ('发展', 1253),
  ('生产', 1231),
  ('进行', 1134),
  ('一个', 1130),
  ('国家', 1065),
  ('问题', 1059),
  ('工作', 1036),
  ('美国', 1012),
  ('反对', 873),
  ('帝国主义', 860),
  ('越南', 844),
  ('美帝国主义', 801),
  ('必须', 800),
  ('世界', 776),
  ('自己', 746),
  ('群众', 741),
  ('领导', 729),
  ('这个', 720),
  ('政治', 704),
  ('运动', 673),
  ('建设', 665),
  ('农业', 651),
  ('力量', 644),
  ('一切', 637),
  ('我国', 636),
  ('民族', 633),
  ('没有', 619),
  ('思想', 598),
  ('已经', 581),
  ('方面', 570),
  ('印度尼西亚', 569),
  ('中国', 553),
  ('农村', 546),
  ('这些', 543),
  ('经济', 536),
  ('这种', 530),
  ('但是', 522),
  ('胜利', 515),
  ('这样', 510),
  ('就是', 506),
  ('修正主义', 496),
  ('政策', 495),
  ('会议', 473),
  ('组织', 467),
  ('同志', 449),
  ('起来', 449),
  ('共产党', 445),
  ('加强', 444),
  ('要求', 425),
  ('战争', 425),
  ('支持', 420),
  ('农民', 419),
  ('南方', 407),
  ('侵略', 406),
  ('日本', 403),
  ('经验', 402),
  ('团结', 402),
  ('任务', 398),
  ('取得'

### Plot Changes in Frequencies of Keywords

In [201]:
# Function that calculates frequencies during 1965-2002 of a given word
def cal_freq(kwd, counts=counters):
    freqs = []
    for k, v in counts.items():
        tol = sum([pair[1] for pair in counts[k]])
        words = [pair[0] for pair in counts[k]]
        cnt = [pair[1] for pair in counts[k]]
        if kwd in words:
            idx = words.index(kwd)
            freqs.append(cnt[idx] / tol)
        else:
            freqs.append(0)
    return freqs

In [208]:
# Plot the frequencies
kwds = ["计划经济", "市场经济", "商品经济", "出口贸易", "沿海经济"]
F = {"year": range(1965, 2003)}
for kwd in kwds:
    F[kwd] = cal_freq(kwd)
df = pd.DataFrame(F)
df.set_index(["year"])
df

,year,计划经济,市场经济,商品经济,出口贸易,沿海经济
0,1965,0.000000,0.000000,0.000000,0.000005,0.000000
1,1966,0.000005,0.000000,0.000000,0.000000,0.000000
2,1967,0.000007,0.000000,0.000000,0.000000,0.000000
3,1968,0.000022,0.000000,0.000011,0.000000,0.000000
4,1969,0.000022,0.000000,0.000000,0.000000,0.000000
5,1970,0.000099,0.000000,0.000000,0.000000,0.000000
6,1971,0.000053,0.000000,0.000006,0.000000,0.000000
7,1972,0.000034,0.000000,0.000000,0.000000,0.000000
8,1973,0.000051,0.000000,0.000000,0.000000,0.000000
9,1974,0.000020,0.000000,0.000000,0.000000,0.000000
